In [1]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
#import Excel
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime

In [61]:
pd.__version__

u'0.20.1'

In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Dont run this block for now

In [ ]:
def aggChargeOffs():
    agg_df = pd.DataFrame()
    for i in range(0, len(dfTapes[deal].index)):
        if dfTapes[deal][i] == 'None':
            break
        else:
            path = directory + '\\' + shelf + '\\' + deal + '\\' + dfTapes[deal][i]
            
            df = pd.read_csv(path, memory_map=True, low_memory=False)
            df = df[~df.zeroBalanceCode.isin(['-'])]
            #df['Month'] = 
            agg_df = pd.concat([agg_df, df])
    
    return agg_df
aggChargeOffs()


*______*

In [3]:
wb = xw.Book('Salt.xlsm')
setup = wb.sheets('Setup')
output = wb.sheets('Output')
strats = wb.sheets('Strat Tables')
performance = wb.sheets('Performance')

# Path For the Current deal that is selected based on Setup
directory = setup['Setup.Directory'].value
shelf = setup['Setup.Shelf'].value
deal = setup['Setup.Deal'].value
report = setup['Setup.Report'].value
num_Deals = setup['Setup.Num_Deals'].value
max_Report = setup['Setup.Max_Report_Num'].value

In [4]:
tapes = np.array(setup['Tapes'].value)
tapes = tapes[0:int(max_Report),0:int(num_Deals)]

In [5]:
dfTapes = pd.DataFrame(data=tapes[1:,1:], columns=tapes[0,1:])
dfTapes.head()

,AMCAR 2017-1,AMCAR 2017-2,AMCAR 2017-3,SDART 2017-1,SDART 2017-2,SDART 2017-3
0,2017\01 January\2017.03.23_AmeriCredit Automob...,2017\03 March\2017.05.02_AmeriCredit Automobil...,2017\06 June\2017.07.01_AmeriCredit Automobile...,2017\01 January\2017.02.15_Santander Drive Aut...,2017\04 April\2017.05.18_Santander Drive Auto ...,2017\08 August\2017.09.07_Santander Drive Auto...
1,2017\02 February\2017.03.23_AmeriCredit Automo...,2017\04 April\2017.06.23_AmeriCredit Automobil...,2017\07 July\2017.08.25_AmeriCredit Automobile...,2017\02 February\2017.03.15_Santander Drive Au...,2017\05 May\2017.06.15_Santander Drive Auto Re...,2017\09 September\2017.10.16_Santander Drive A...
2,2017\03 March\2017.04.24_AmeriCredit Automobil...,2017\05 May\2017.07.24_AmeriCredit Automobile ...,2017\08 August\2017.09.25_AmeriCredit Automobi...,2017\03 March\2017.04.17_Santander Drive Auto ...,2017\06 June\2017.07.17_Santander Drive Auto R...,None
3,2017\04 April\2017.05.23_AmeriCredit Automobil...,2017\06 June\2017.08.23_AmeriCredit Automobile...,None,2017\04 April\2017.05.15_Santander Drive Auto ...,2017\07 July\2017.08.15_Santander Drive Auto R...,None
4,2017\05 May\2017.06.23_AmeriCredit Automobile ...,2017\08 August\2017.09.25_AmeriCredit Automobi...,None,2017\05 May\2017.06.15_Santander Drive Auto Re...,2017\08 August\2017.09.15_Santander Drive Auto...,None


In [10]:
path = directory + '\\' + shelf + '\\' + deal + '\\' + dfTapes[deal][8]

In [171]:
df = pd.read_csv(path, memory_map=True, low_memory=False)
len(df.index)

66797

In [102]:
df['obligorGeographicLocation'].unique()

array(['AZ', 'TX', 'WA', 'FL', 'IL', 'VA', 'CA', 'NY', 'NC', 'UT', 'OH',
       'NV', 'DC', 'MS', 'MD', 'NM', 'GA', 'IN', 'MI', 'ME', 'MO', 'OK',
       'TN', 'AR', 'AL', 'NJ', 'CT', 'LA', 'SC', 'WV', 'KY', 'DE', 'IA',
       'CO', 'WI', 'VT', 'HI', 'NH', 'OR', 'RI', 'KS', 'MN', 'WY', 'SD',
       'NE', 'ID', 'ND', 'MT', 'AK'], dtype=object)

In [179]:
df = df[df.zeroBalanceCode.isin(['-'])]
print(len(df.index))
df['vehicleNewUsedCode'] = pd.to_numeric(df['vehicleNewUsedCode'], errors='coerce')
df['obligorIncomeVerificationLevelCode'] = pd.to_numeric(df['obligorIncomeVerificationLevelCode'], errors='coerce')
df['obligorEmploymentVerificationCode'] = pd.to_numeric(df['obligorEmploymentVerificationCode'], errors='coerce')
cs = df['obligorCreditScore'][~df.obligorCreditScore.isin(['-'])]
cs['obligorCreditScore'] = pd.to_numeric(cs, errors='coerce')
print(len(cs.index))

66797
57204


In [181]:
WA = lambda x: np.average(x, axis=0, weights = df.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100
df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio

In [186]:
a = OrderedDict([
    ('pctDeal', ['sum'])
])

income = {
    1:'Not Stated, Not Verified',
    2:'Stated, Not Verified',
    3:'Stated, Verified',
    4:'Stated, Verified: 12 M',
    5:'Stated, Verified: 24 M'
}

job =  {
    1:'Not Stated, Not Verified',
    2:'Stated, Not Verified',
    3:'Stated, Third Party Verified'
}

newUsed = {
    1:'New Vehicle',
    2:'Used Vehicle'
}

avgDict = {
    0:'Credit Score at Origination',
    1:'Remaining Term to Maturity',
    2:'APR at Origination',
    3:'Payment to Income'
}

wAvg = {
    'Weighted Average':[WA(cs['obligorCreditScore']),
                                WA(df['remainingTermToMaturityNumber']),
                                WA(df['originalInterestRatePercentage']),
                                WA(df['paymentToIncomePercentage'])]
}

In [189]:
WAdf = pd.DataFrame(data=wAvg).rename(index=avgDict)

nonSubvented = df[df['subvented'] == '0']['pctDeal'].sum()
subvented = df[df['subvented'] != '0']['pctDeal'].sum()

newUsed = df[['pctDeal']].groupby(df['vehicleNewUsedCode']).sum().rename(index=newUsed)
topManufacturer = (df[['pctDeal']].groupby(df['vehicleManufacturerName']).sum()).sort_values(['pctDeal'], ascending=False).head(5)
topModel = (df[['pctDeal']].groupby(df['vehicleModelName']).sum()).sort_values(['pctDeal'], ascending=False).head(5)
topLocation = (df[['pctDeal']].groupby(df['obligorGeographicLocation']).sum()).sort_values(['pctDeal'], ascending=False).head(5)

incomeVerify = df[['pctDeal']].groupby(df['obligorIncomeVerificationLevelCode']).sum().rename(index=income)
jobVerify = df[['pctDeal']].groupby(df['obligorEmploymentVerificationCode']).sum().rename(index=job)
